# Deep Fake Detection


## 0.1 Load necessary libraries

In [1]:
%%capture

# standard libraries
import pandas as pd
import numpy as np
from numpy import *
import sys

# for visuals
import matplotlib.pyplot as plt 
import matplotlib

# handling files
import zipfile
import os
import fnmatch
import regex as re

# dealing with images
import skimage.color
import skimage.exposure
import skimage.io
import skimage.util
import skimage
import cv2
from skimage import img_as_ubyte
from PIL import Image
from sklearn import *
from sklearn.model_selection import train_test_split
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.utils import compute_class_weight

#for cleaning up memory
import gc

#for deleting files
import shutil

#for shuffeling the data
import random
from sklearn.utils import shuffle

#for model visuals
!pip install keras==2.8.*

#for deep learning
!pip install --upgrade tensorflow==2.8
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SimpleRNN
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers.serialization import populate_deserializable_objects
from keras.regularizers import l2


#modifying images
!pip install cvzone
!pip install mediapipe
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation
import cv2
segmentor = SelfiSegmentation()


In [11]:
# skip if not connected with GOOGLE COLAB

#note I had to run the file on google colab since my processor was too weak
#from google.colab import drive

#connect to google drive
#drive.mount("/content/gdrive", force_remount=True)
#%cd gdrive/My\ Drive/Colab_Notebooks/ML/

## Define Data Loading and Evaluation Functions

In [3]:

def load_color_image(path, imagesize):
    """ simple function
    to open an image from path as color image"""
    # open file
    image = cv2.imread(path)
    # change color
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    # remove the background
    image = segmentor.removeBG(image, (255, 255, 255), threshold=0.5)
    # change contrast
    # if we set alpha > 1 we will incrase the contrast
    image = cv2.convertScaleAbs(image, alpha=1.2, beta=0)
    # sharpen the imamge
    image = cv2.filter2D(image, -1, np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]))
    # change size
    image = cv2.resize(image, imagesize)
    # return normalized image
    return image/255
      
def load_test_data(imagesize, home_path = './data/'):
    """
    The main functionality to load images as numpy array given a datapath
    Paramater imagesize is a 1x2 array indicating how big the
    loaded image should be, bw meaning if black and white pictures should be loaded.
    """

    testX = []
    testY = []

    for data_sub_set in ['test']:
      for encoded_class, class_name in enumerate(['original', 'manipulated']):
        for path, subdir, files in os.walk(home_path + data_sub_set + '/' + class_name + '/'):
          for name in files:
                loaded_image = load_color_image(os.path.join(path, name), imagesize)
                testX.append(loaded_image)
                testY.append(encoded_class)

    # convert to numpy arrays
    testX = np.array(testX, dtype = 'float32')
    testY = np.array(testY, dtype = 'int32')

    # returning scaled images
    return testX, testY
    
imagesize=(180,180)
def test(Model, Data_Path = './data/'):
  
    """ Function to load test data 
     make prediction on test data and to output the corresponding accuracy
    """

    testX, testY = load_test_data(imagesize, Data_Path)
    testX, testY = shuffle(testX, testY)

    X = testX
    Y = testY
    
    print("----------------------------------------------------------------")
    print("Evaluation of Test  Set")

    #doing the prediction
    Y_hat = Model.predict(testX)
    #predYtest[Model_Name] = Y_hat

    if "keras.engine" in str(type(Model)):
      Y_hat = np.array(np.round_(Y_hat, decimals = 0, out = None), dtype="int32")
      Y_hat = np.array([x[0] for x in Y_hat.tolist()], dtype="int32")
    
    #outputing accuracy, recall precision and AUC
    accuracy = metrics.accuracy_score(Y, Y_hat)
    recall = metrics.recall_score(Y, Y_hat)
    precision = metrics.precision_score(Y, Y_hat)
    AUC = metrics.roc_auc_score(Y, Y_hat)
    print("Accuracy Score: {:.2f}".format(accuracy))
    print("Recall Score: {:.2f}".format(recall))
    print("Precision Score: {:.2f}".format(precision))
    print("AUC: {:.2f}".format(AUC))
        
    print("----------------------------------------------------------------")
    return accuracy, recall, precision, AUC

## Define Model

In [8]:
class RandomBrightnessChange(tf.keras.layers.Layer):
    def __init__(self, 
                 brightness_delta=[-0.1, 0.1], **kwargs):
        super(RandomBrightnessChange, self).__init__(**kwargs)
        self.brightness_delta = brightness_delta
    
    def call(self, images, training=None):
        if not training:
            return images
        
        brightness = np.random.uniform(
            self.brightness_delta[0], self.brightness_delta[1])
        images = tf.image.adjust_brightness(images, brightness)
        images = tf.clip_by_value(images, 0, 1)
        return images

        
augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.03, input_shape = (imagesize[0], imagesize[1], 3)),
    tf.keras.layers.RandomContrast(0.2),
    RandomBrightnessChange()
    
  ]
)

filter=16
model = tf.keras.Sequential([
      augmentation,
      tf.keras.layers.Conv2D(filter, kernel_size=3, activation="tanh"),
      tf.keras.layers.Conv2D(filter, kernel_size=3), 
      tf.keras.layers.LeakyReLU(alpha=0.05),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.3),

      tf.keras.layers.Conv2D(filter*2, kernel_size=4, activation="tanh", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-10, l2=1e-9)),
      tf.keras.layers.Conv2D(filter*2, kernel_size=4, kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-10, l2=1e-9)),
      tf.keras.layers.LeakyReLU(alpha=0.05),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.3),

      tf.keras.layers.Conv2D(filter*2*2, kernel_size=5, activation="tanh", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-10, l2=1e-9)),
      tf.keras.layers.Conv2D(filter*2*2, kernel_size=5),
      tf.keras.layers.LeakyReLU(alpha=0.05),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.3),

      tf.keras.layers.Conv2D(filter*2*2*2*2, kernel_size=5, activation="tanh", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-10, l2=1e-9)),
      tf.keras.layers.Conv2D(filter*2*2*2*2, kernel_size=5, kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-10, l2=1e-9)),
      tf.keras.layers.LeakyReLU(alpha=0.05),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dropout(0.3),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-7, l2=1e-6)),
      tf.keras.layers.Dropout(0.7),

      tf.keras.layers.Dense(1, activation="sigmoid")
    ])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00015, decay=0.00001), loss = "binary_crossentropy", metrics=['accuracy']) 



/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Load Trained Weights

In [12]:
model.load_weights("./weights.h5")

## Make Predictions

In [ ]:
test(model, "./data/")